In [0]:
spark.sql("USE globalretail_silver")
spark.sql("""
CREATE TABLE IF NOT EXISTS silver_orders (
    transaction_id STRING,
    customer_id STRING,
    product_id STRING,
    quantity INT,
    total_amount DOUBLE,
    transaction_date DATE,
    payment_method STRING,
    store_type STRING,
    order_status STRING,
    last_updated TIMESTAMP
)
USING DELTA
""")

DataFrame[]

In [0]:
# Get the last processed timestamp from silver layer
last_processed_df = spark.sql("SELECT MAX(last_updated) as last_processed FROM silver_orders")
last_processed_timestamp = last_processed_df.collect()[0]['last_processed']

if last_processed_timestamp is None:
    last_processed_timestamp = "1900-01-01T00:00:00.000+00:00"

In [0]:
# Create a temporary view of incremental bronze data
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW bronze_incremental_orders AS
SELECT *
FROM globalretail_bronze.bronze_transactions WHERE ingestion_timestamp > '{last_processed_timestamp}'

""")

DataFrame[]

In [0]:
spark.sql("select * from bronze_incremental_orders").show()

+--------------+-----------+----------+--------+------------+-------------------+--------------+--------------+--------------------+
|transaction_id|customer_id|product_id|quantity|total_amount|   transaction_date|payment_method|    store_type| ingestion_timestamp|
+--------------+-----------+----------+--------+------------+-------------------+--------------+--------------+--------------------+
|     TRX000001|        802|       425|       1|       363.4|2020-07-27 00:00:00|    Debit Card|Physical Store|2024-07-19 04:40:...|
|     TRX000002|        858|       280|       6|      758.18|2022-08-10 00:00:00|   Credit Card|Physical Store|2024-07-19 04:40:...|
|     TRX000003|        658|       694|       9|      748.66|2020-05-22 00:00:00| Bank Transfer|        Online|2024-07-19 04:40:...|
|     TRX000004|        516|       930|       4|      933.78|               NULL| Bank Transfer|Physical Store|2024-07-19 04:40:...|
|     TRX000005|        368|       104|      10|      137.28|2022-06-

Data Transformations:
   - Quantity and total_amount normalization (setting negative values to 0)
   - Date casting to ensure consistent date format
   - Order status derivation based on quantity and total_amount

Data Quality Checks: We filter out records with null transaction dates, customer IDs, or product IDs.


In [0]:
spark.sql("""
CREATE OR REPLACE TEMPORARY VIEW silver_incremental_orders AS
SELECT
    transaction_id,
    customer_id,
    product_id,
    CASE 
        WHEN quantity < 0 THEN 0 
        ELSE quantity 
    END AS quantity,
    CASE 
        WHEN total_amount < 0 THEN 0 
        ELSE total_amount 
    END AS total_amount,
    CAST(transaction_date AS DATE) AS transaction_date,
    payment_method,
    store_type,
    CASE
        WHEN quantity = 0 OR total_amount = 0 THEN 'Cancelled'
        ELSE 'Completed'
    END AS order_status,
    CURRENT_TIMESTAMP() AS last_updated
FROM bronze_incremental_orders
WHERE transaction_date IS NOT NULL
  AND customer_id IS NOT NULL
  AND product_id IS NOT NULL
""")

DataFrame[]

In [0]:
spark.sql("select * from silver_incremental_orders").show()

+--------------+-----------+----------+--------+------------+----------------+--------------+--------------+------------+--------------------+
|transaction_id|customer_id|product_id|quantity|total_amount|transaction_date|payment_method|    store_type|order_status|        last_updated|
+--------------+-----------+----------+--------+------------+----------------+--------------+--------------+------------+--------------------+
|     TRX000001|        802|       425|       1|       363.4|      2020-07-27|    Debit Card|Physical Store|   Completed|2024-07-19 05:39:...|
|     TRX000002|        858|       280|       6|      758.18|      2022-08-10|   Credit Card|Physical Store|   Completed|2024-07-19 05:39:...|
|     TRX000003|        658|       694|       9|      748.66|      2020-05-22| Bank Transfer|        Online|   Completed|2024-07-19 05:39:...|
|     TRX000005|        368|       104|      10|      137.28|      2022-06-24|        PayPal|Physical Store|   Completed|2024-07-19 05:39:...|

In [0]:
spark.sql("""
MERGE INTO silver_orders target
USING silver_incremental_orders source
ON target.transaction_id = source.transaction_id
WHEN MATCHED THEN
    UPDATE SET *
WHEN NOT MATCHED THEN
    INSERT *
""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
select * from silver_orders

transaction_id customer_id product_id quantity total_amount transaction_date payment_method store_type order_status last_updated TRX000001 802 425 1 363.4 2020-07-27 Debit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000002 858 280 6 758.18 2022-08-10 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000003 658 694 9 748.66 2020-05-22 Bank Transfer Online Completed 2024-07-19T05:40:03.428Z TRX000005 368 104 10 137.28 2022-06-24 PayPal Physical Store Completed 2024-07-19T05:40:03.428Z TRX000006 606 409 6 556.88 2020-10-14 Credit Card Online Completed 2024-07-19T05:40:03.428Z TRX000007 535 487 7 246.29 2023-08-19 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000008 87 630 8 0.0 2020-04-18 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000009 383 617 10 405.05 2021-02-13 Cash Physical Store Completed 2024-07-19T05:40:03.428Z TRX000010 725 938 1 410.03 2020-03-18 PayPal Online Completed 2024-07-19T05:40:03.428Z TRX000011 687 457 8 269.88 2020-06-27 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000012 746 552 10 801.14 2021-10-20 Bank Transfer Physical Store Completed 2024-07-19T05:40:03.428Z TRX000013 16 681 5 421.51 2021-06-01 PayPal Online Completed 2024-07-19T05:40:03.428Z TRX000014 458 811 4 262.38 2023-12-14 Bank Transfer Physical Store Completed 2024-07-19T05:40:03.428Z TRX000015 391 84 2 867.84 2021-03-26 Debit Card Online Completed 2024-07-19T05:40:03.428Z TRX000016 899 194 10 212.75 2023-10-13 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000017 296 646 10 498.83 2023-08-26 Debit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000018 483 658 10 91.09 2023-06-05 PayPal Online Completed 2024-07-19T05:40:03.428Z TRX000019 783 406 9 610.69 2023-06-27 PayPal Physical Store Completed 2024-07-19T05:40:03.428Z TRX000020 984 522 7 804.71 2020-10-14 Credit Card Online Completed 2024-07-19T05:40:03.428Z TRX000021 462 666 10 988.44 2023-04-07 PayPal Online Completed 2024-07-19T05:40:03.428Z TRX000022 253 455 2 876.7 2021-12-29 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000023 810 720 3 44.16 2023-07-02 Debit Card Online Completed 2024-07-19T05:40:03.428Z TRX000024 931 870 5 366.96 2023-01-15 Bank Transfer Online Completed 2024-07-19T05:40:03.428Z TRX000025 489 392 4 854.16 2023-03-01 Bank Transfer Physical Store Completed 2024-07-19T05:40:03.428Z TRX000026 800 404 7 547.84 2022-10-21 Bank Transfer Physical Store Completed 2024-07-19T05:40:03.428Z TRX000027 560 5 6 491.86 2023-12-11 Credit Card Online Completed 2024-07-19T05:40:03.428Z TRX000028 921 460 5 739.54 2023-10-18 Cash Online Completed 2024-07-19T05:40:03.428Z TRX000029 515 616 9 319.22 2020-04-09 PayPal Online Completed 2024-07-19T05:40:03.428Z TRX000030 973 708 1 737.1 2023-01-05 Bank Transfer Physical Store Completed 2024-07-19T05:40:03.428Z TRX000031 903 729 5 929.07 2020-12-21 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000032 765 116 1 849.13 2021-10-07 Credit Card Physical Store Completed 2024-07-19T05:40:03.428Z TRX000033 727 806 8 890.95 2022-07-07 PayPal Physical Store Completed 2024-07-19T05:40:03.428Z TRX000034 327 14 3 756.04 2021-02-08 Bank Transfer Physical Store Completed 2024-07-19T05:40:03.428Z TRX000035 941 46 4 782.36 2021-02-09 Cash Online Completed 2024-07-19T05:40:03.428Z TRX000036 463 978 9 320.96 2022-11-23 Bank Transfer Online Completed 2024-07-19T05:40:03.428Z TRX000037 419 57 3 980.32 2020-05-13 Bank Transfer Physical Store Completed 2024-07-19T05:40:03.428Z TRX000038 637 760 9 28.94 2022-06-23 Credit Card Online Completed 2024-07-19T05:40:03.428Z TRX000039 158 356 1 284.36 2020-05-28 PayPal Online Completed 2024-07-19T05:40:03.428Z TRX000040 557 933 8 154.84 2020-01-19 Cash Physical Store Completed 2024-07-19T05:40:03.428Z TRX000041 227 41 2 263.01 2023-10-05 PayPal Online Completed 2024-07-19T05:40:03.428Z TRX000042 824 972 3 233.21 2020-06-12 PayPal Online Completed 2024-07-19T05:40:03.428Z T